# Kelvin-Helmholtz Instability Visualization

This notebook demonstrates how to visualize 2D Kelvin-Helmholtz instability simulations.

## Contents
1. Loading 2D simulation data
2. Creating particle scatter plots
3. Vorticity analysis
4. Conservation checking
5. Creating animations

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import sys
sys.path.append('..')

from readers import SimulationReader
from conservation import ConservationAnalyzer
from plotting import ParticlePlotter

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Simulation Data

In [ ]:
# Path to simulation results
results_dir = Path('../../results/khi')  # Update this path

# Load all snapshots
reader = SimulationReader(results_dir)
snapshots = reader.read_all_snapshots()

print(f"Loaded {len(snapshots)} snapshots")
print(f"Time range: {snapshots[0].time:.4f} to {snapshots[-1].time:.4f}")
print(f"Dimension: {snapshots[0].dim}D")
print(f"Number of particles: {snapshots[0].n_particles}")

## 2. Visualize Density Field

Plot the density field at different times to see the KHI development.

In [ ]:
# Select snapshots at different times
n_snapshots = len(snapshots)
indices = [0, n_snapshots//4, n_snapshots//2, 3*n_snapshots//4, -1]

plotter = ParticlePlotter(figsize=(15, 12))

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flat

for i, idx in enumerate(indices):
    snap = snapshots[idx]
    ax, scatter = plotter.plot_2d_scatter(snap, quantity='dens', ax=axes[i])
    axes[i].set_title(f't = {snap.time:.4f}')
    plt.colorbar(scatter, ax=axes[i], label='Density')

# Remove extra subplot
fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

## 3. Velocity Field Visualization

Show the velocity field with arrows to see the shear flow and vortex development.

In [ ]:
# Plot velocity field at final time
final_snap = snapshots[-1]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Density with velocity overlay
ax, scatter = plotter.plot_2d_scatter(final_snap, quantity='dens', ax=axes[0])

# Subsample particles for quiver plot (too many arrows otherwise)
step = max(1, final_snap.n_particles // 1000)
x = final_snap.pos[::step, 0]
y = final_snap.pos[::step, 1]
vx = final_snap.vel[::step, 0]
vy = final_snap.vel[::step, 1]

axes[0].quiver(x, y, vx, vy, alpha=0.5, scale=5)
axes[0].set_title(f'Density and Velocity at t = {final_snap.time:.4f}')
plt.colorbar(scatter, ax=axes[0], label='Density')

# Velocity magnitude
vel_mag = np.sqrt(final_snap.vel[:, 0]**2 + final_snap.vel[:, 1]**2)
scatter2 = axes[1].scatter(final_snap.pos[:, 0], final_snap.pos[:, 1], 
                           c=vel_mag, s=5, cmap='plasma')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_aspect('equal')
axes[1].set_title(f'Velocity Magnitude at t = {final_snap.time:.4f}')
plt.colorbar(scatter2, ax=axes[1], label='|v|')

plt.tight_layout()
plt.show()

## 4. Vorticity Analysis

Compute and visualize the vorticity field to identify vortices.

In [ ]:
def compute_vorticity_2d(snapshot):
    """
    Simple vorticity estimate from velocity gradients.
    Note: This is approximate - proper SPH vorticity uses kernel gradients.
    """
    # Create grid
    nx, ny = 100, 100
    x_edges = np.linspace(snapshot.pos[:, 0].min(), snapshot.pos[:, 0].max(), nx+1)
    y_edges = np.linspace(snapshot.pos[:, 1].min(), snapshot.pos[:, 1].max(), ny+1)
    
    # Bin particles and compute average velocity in each cell
    vx_grid = np.zeros((ny, nx))
    vy_grid = np.zeros((ny, nx))
    count = np.zeros((ny, nx))
    
    for i in range(snapshot.n_particles):
        ix = np.searchsorted(x_edges, snapshot.pos[i, 0]) - 1
        iy = np.searchsorted(y_edges, snapshot.pos[i, 1]) - 1
        if 0 <= ix < nx and 0 <= iy < ny:
            vx_grid[iy, ix] += snapshot.vel[i, 0]
            vy_grid[iy, ix] += snapshot.vel[i, 1]
            count[iy, ix] += 1
    
    # Average
    mask = count > 0
    vx_grid[mask] /= count[mask]
    vy_grid[mask] /= count[mask]
    
    # Compute vorticity: omega_z = dv_y/dx - dv_x/dy
    dx = x_edges[1] - x_edges[0]
    dy = y_edges[1] - y_edges[0]
    
    dvy_dx = np.gradient(vy_grid, dx, axis=1)
    dvx_dy = np.gradient(vx_grid, dy, axis=0)
    
    vorticity = dvy_dx - dvx_dy
    
    return vorticity, x_edges, y_edges

# Compute vorticity at final time
vorticity, x_edges, y_edges = compute_vorticity_2d(final_snap)

# Plot
fig, ax = plt.subplots(figsize=(12, 8))

X, Y = np.meshgrid(x_edges[:-1], y_edges[:-1])
im = ax.pcolormesh(X, Y, vorticity, cmap='RdBu_r', shading='auto')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_aspect('equal')
ax.set_title(f'Vorticity at t = {final_snap.time:.4f}')
plt.colorbar(im, ax=ax, label=r'$\omega_z$')

plt.show()

## 5. Conservation Analysis

In [ ]:
# Compute conservation
analyzer = ConservationAnalyzer()
report = analyzer.analyze_snapshots(snapshots)

# Print summary
report.print_summary()

In [ ]:
# Plot conservation errors
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Conservation Errors - KHI', fontsize=16)

# Mass
axes[0, 0].plot(report.time, report.mass_error)
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Relative error')
axes[0, 0].set_title('Mass Conservation')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(0, color='k', linestyle='--', alpha=0.3)

# Momentum
axes[0, 1].plot(report.time, report.momentum_error)
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Relative error')
axes[0, 1].set_title('Momentum Conservation')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(0, color='k', linestyle='--', alpha=0.3)

# Angular momentum (2D)
if report.angular_momentum_error is not None:
    axes[1, 0].plot(report.time, report.angular_momentum_error)
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('Relative error')
    axes[1, 0].set_title('Angular Momentum Conservation')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].axhline(0, color='k', linestyle='--', alpha=0.3)

# Energy
axes[1, 1].plot(report.time, report.energy_error)
axes[1, 1].set_xlabel('Time')
axes[1, 1].set_ylabel('Relative error')
axes[1, 1].set_title('Energy Conservation')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Animation

Create an animation of the KHI evolution.

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots(figsize=(12, 8))

def animate(i):
    ax.clear()
    snap = snapshots[i]
    scatter = ax.scatter(snap.pos[:, 0], snap.pos[:, 1], 
                        c=snap.dens, s=10, cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal')
    ax.set_title(f'KHI at t = {snap.time:.4f}')
    ax.set_xlim([snapshots[0].pos[:, 0].min(), snapshots[0].pos[:, 0].max()])
    ax.set_ylim([snapshots[0].pos[:, 1].min(), snapshots[0].pos[:, 1].max()])
    return scatter,

anim = FuncAnimation(fig, animate, frames=len(snapshots), interval=100, blit=False)

# Display
HTML(anim.to_jshtml())

## Summary

This tutorial demonstrated:
- Loading and visualizing 2D KHI simulations
- Plotting density and velocity fields
- Computing and visualizing vorticity
- Checking conservation properties
- Creating animations

### Next Steps
- Try different density ratios
- Vary the perturbation amplitude
- Compare different SPH methods
- Analyze growth rates of the instability
- Study resolution dependence